In [2]:
import requests
from bs4 import BeautifulSoup

import pdb



In [3]:
import multiprocessing as mp
import time

In [35]:
DPG_MAX = 11858

class ScrapingViePubliqueDiscours(object):
    def __init__(self):
        self.dind_outfp = '/Users/yukatherin/Downloads/vie-publique-discours/discours_index.tsv'  # Format: href\ttitle\tdate
        
        self.durl_pgfmt = 'https://vie-publique.fr/discours?page={}'
    
    @classmethod
    def parse_href(cls, ll):
        """
        Input:'<a class="link-multiple" href="/discours/278995-entretien-jean-baptiste-lemoyne-11032021-tourime-en-montagne" rel="bookmark">'
        Output:'/discours/278995-entretien-jean-baptiste-lemoyne-11032021-tourime-en-montagne'
        """
        s = BeautifulSoup(ll, "html.parser")
        return s.a['href']
    
    @classmethod
    def parse_date(cls, ll):
        """
        Input:'<span class="field field--name-field-date-prononciation-discour field--type-datetime field--label-hidden field__item"><time class="datetime" datetime="2021-03-11T12:00:00Z">11 mars 2021</time>'
        Ouput:'2021-03-11T12:00:00Z'
        """
        s = BeautifulSoup(ll, "html.parser")
        return s.time['datetime']

    @classmethod
    def parse_title(cls, ll):
        """
        Input:'<span class="field--name-title">Entretien de M. Clément Beaune, secrétaire d'Etat aux affaires européennes, avec France Inter le 11 mars 2021, sur les droits des personnes LGBT en Pologne et sur la vaccination contre le coronavirus dans l'Union européenne.</span>'
        Output:'Entretien de M. Clément Beaune, secrétaire d'Etat aux affaires européennes, avec France Inter le 11 mars 2021, sur les droits des personnes LGBT en Pologne et sur la vaccination contre le coronavirus dans l'Union européenne.'
        """
        s = BeautifulSoup(ll, "html.parser")
        return s.select_one("span").text

        
    def __call__(self, pg):
        durl = self.durl_pgfmt.format(pg)
        dpage = requests.get(durl)
        sp = BeautifulSoup(dpage.content, 'html.parser')
        sp_inmem = str(sp).split('\n')
                
        for i, ll in enumerate(sp_inmem):
            if ll.startswith('<span class="field--name-title">'):
                try:
                    ll_href = ScrapingViePubliqueDiscours.parse_href(sp_inmem[i - 1])
                    ll_dt = ScrapingViePubliqueDiscours.parse_date(sp_inmem[i + 5])
                    ll_title = ScrapingViePubliqueDiscours.parse_title(sp_inmem[i])
                    yield ("{}\t{}\t{}\n".format(ll_href, ll_dt, ll_title))
                except TypeError as e:
                    continue
    
    def test():
        print(
            ScrapingViePubliqueDiscours.parse_href(
                '<a class="link-multiple" href="/discours/278995-entretien-jean-baptiste-lemoyne-11032021-tourime-en-montagne" rel="bookmark">'
            )        
        )
        print(
            ScrapingViePubliqueDiscours.parse_date(
                '<span class="field field--name-field-date-prononciation-discour field--type-datetime field--label-hidden field__item"><time class="datetime" datetime="2021-03-11T12:00:00Z">11 mars 2021</time>'
            )
        )
        print(
            ScrapingViePubliqueDiscours.parse_title(
                """<span class="field--name-title">Entretien de M. Clément Beaune, secrétaire d'Etat aux affaires européennes, avec France Inter le 11 mars 2021, sur les droits des personnes LGBT en Pologne et sur la vaccination contre le coronavirus dans l'Union européenne.</span>"""
            )
        )

ScrapingViePubliqueDiscours.test()

/discours/278995-entretien-jean-baptiste-lemoyne-11032021-tourime-en-montagne
2021-03-11T12:00:00Z
Entretien de M. Clément Beaune, secrétaire d'Etat aux affaires européennes, avec France Inter le 11 mars 2021, sur les droits des personnes LGBT en Pologne et sur la vaccination contre le coronavirus dans l'Union européenne.


In [36]:
def run_mp_svpd_singlethread():
    svpd = ScrapingViePubliqueDiscours()
    with open(svpd.dind_outfp, 'a') as f:
        for pg in range(1, DPG_MAX + 1):
            if pg % 50 == 0:
                print("{} {}".format(pg, time.time()))
            for out_line in svpd(pg):
                f.write(out_line)
run_mp_svpd_singlethread()

50 1616061504.652097
100 1616061751.225955
150 1616062004.213496
200 1616062265.26971
250 1616062523.516971
300 1616062774.190169
350 1616063033.1148238
400 1616063300.722822
450 1616063557.2608068
500 1616063810.155684
550 1616064064.0027509
600 1616064319.074197
650 1616064569.641917
700 1616064820.825654
750 1616065077.090562
800 1616065333.123403
850 1616065579.905489
900 1616065834.060978
950 1616066290.759767
1000 1616066552.599392
1050 1616066807.255763
1100 1616067053.013542
1150 1616067297.645747
1200 1616067542.728829
1250 1616067796.229464
1300 1616068045.047734
1350 1616068289.247555
1400 1616068543.829777
1450 1616068798.191605
1500 1616069057.051459
1550 1616069308.145907
1600 1616069564.027942
1650 1616069818.153137
1700 1616070072.333235
1750 1616070329.973279
1800 1616070595.889359
1850 1616070846.775025
1900 1616071104.190926
1950 1616071372.382869
2000 1616071639.7526631
2050 1616071892.576616
2100 1616072142.320792
2150 1616072399.959743
2200 1616072664.34862
2250 1

In [38]:
# This code does not work.

def run_mp_svpd():

    # Init listener
    svpd = ScrapingViePubliqueDiscours()
    f = open(svpd.dind_outfp, 'w')

    def mp_listener(q):
        with open(svpd.dind_outfp, 'w') as f:
            while 1:
                m = q.get()
                if m == "kill":
                    break
                f.write(m)
                f.flush()

    # Init manager and pool
    manager = mp.Manager()
    q = manager.Queue()
    pool = mp.Pool(processes=mp.cpu_count()-1)
    
    # Put listener on async
    watcher = pool.apply_async(mp_listener, (q,))
    
    # Put workers on async
    jobs = []
    for pg in range(1, 5): # TODO
        job = pool.apply_async(svpd, (pg, q))
        jobs.append(job)
    
    # Collect jobs
    for job in jobs:
        job.get()
        
    # Kill listener
    q.put('kill')
    pool.close()
    pool.join()
